In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import time
import random
from openai import OpenAI
from tqdm import tqdm

In [ ]:
!pip install openai

In [ ]:
os.environ["OPENAI_API_KEY"] = ""


def get_response(msg):
    messages = [
        {
            "role": "user",
            "content": msg
        }
    ]

    temperature = random.uniform(0.7, 1.0)
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        temperature=temperature
    )

    output = response.choices[0].message.content
    return output

In [ ]:
df = pd.read_csv('datasets/lb_related_5k.csv')
# df = df[:3]

In [ ]:
df.head()

In [ ]:
# def g(df, col):
#     return df.groupby(col).apply(lambda x: x.sample(1)).reset_index(drop=True)
# result = g(df.copy(), 'rewrite_prompt')
result = df.copy()

In [ ]:
def formatting_prompts_func(examples):
    inst = """
You'll be given an original text and a rewritten text generated by an LLM. The prompt that guided the LLM's changes will be given as well. 
Suppose you don't know the original prompt, your task is to analyze the differences between an original text and a rewritten version generated by an LLM and try to infer it based on the differences you see. 
Provide a detailed explanation of how you arrived at your inference step by step.

**Original Text**:
{}

**Prompt**:
{}

**Rewritten Text**
{}

You should response in the following format:
**Inferred Promp**: ...

**Chain of Thoughts**: ...
"""
    return inst.format(examples['original_text'], examples['rewrite_prompt'], examples['rewritten_text'])

In [ ]:
cot = []
for idx, row in tqdm(result.iterrows(), total=len(result)):
    # 输入提示
    try:
        prompt_parts = formatting_prompts_func(row)
        # 输出回答
        while True:
            try:
                response = get_response(prompt_parts)
                break
            except Exception as e:
                print(e)
                time.sleep(1)

        cot.append(response)
#         print(response)
    except:
        cot.append("")

result['cot'] = cot
result.to_csv('./datasets_with_cot.csv', index=False)

In [ ]:
# result['cot'].values